TODO: Until we get real data, make synthetic data

#### Temporary: Synthetic Data

In [20]:
# Synthetic Data
import numpy as np
from sklearn.datasets import make_classification
X, y = make_classification(random_state=42)
data = np.concatenate((X, y.reshape(-1, 1)), axis = 1)

### Preprocessing

In [21]:
import preprocess
np.random.seed(42)
states = np.random.randint(low = 0, high = 1000000, size=(100,)) # numpy array with our 100 random states
data_splits = {}
for rst in states:
    data_splits[rst] = preprocess.preprocess(data, rst)

### Modeling

In [22]:
import modeling as models
model_names = {"dt", "rf", "gb", "xgb", "lgb", "et", "ab", "lr", "lr1", "lr2", "lre", "lsv", "nlsv", "knn", "lda", "gnb", "mlp"}


model_res = {} # key as model type, then sub dictionary with rst as key and predicted classes & model itself ex: model_res["dt"][rst] --> (y_pred_dt, dt) 
for model in model_names:
    model_res[model] = {}

for state in data_splits.keys():    
    model_res["dt"][state] = models.model_decisiontree(*data_splits[state]) # decision tree
    model_res["rf"][state] = models.model_randomforest(*data_splits[state]) # random forest
    model_res["gb"][state] = models.model_gradientboosting(*data_splits[state]) # grad boosting
    model_res['xgb'][state] = models.model_extremegb(*data_splits[state])
    model_res["lgb"][state] = models.model_lightgb(*data_splits[state])
    model_res["et"][state] = models.model_extratrees(*data_splits[state])
    model_res["ab"][state] = models.model_adaboost(*data_splits[state])
    model_res["lr"][state] = models.model_logisticregression(*data_splits[state])
    model_res["lr1"][state] = models.model_lassoregularization(*data_splits[state])
    model_res["lr2"][state] = models.model_ridgeRegularization(*data_splits[state])
    model_res["lre"][state] = models.model_elasticNetRegularization(*data_splits[state])
    model_res["lsv"][state] = models.model_linearSupportVector(*data_splits[state])
    model_res["nlsv"][state] = models.model_nonLinearSupportVector(*data_splits[state])
    model_res["knn"][state] = models.model_kNearestNeighbor(*data_splits[state])
    model_res["lda"][state] = models.model_linearDiscriminantAnalysis(*data_splits[state])
    model_res["gnb"][state] = models.model_gaussianNaiveBayes(*data_splits[state])
    model_res["mlp"][state] = models.model_multiLayerPerceptron(*data_splits[state])

c:\Users\fawfu\anaconda3\envs\env\Lib\site-packages\sklearn\utils\deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
c:\Users\fawfu\anaconda3\envs\env\Lib\site-packages\sklearn\utils\deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
c:\Users\fawfu\anaconda3\envs\env\Lib\site-packages\sklearn\utils\deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
c:\Users\fawfu\anaconda3\envs\env\Lib\site-packages\sklearn\utils\deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
c:\Users\fawfu\anaconda3\envs\env\Lib\site-packages\sklearn\utils\deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will b

### Evaluation

In [23]:
# basic view for now to see some metrics
from sklearn.metrics import accuracy_score, confusion_matrix
from metrics import getMetrics
model_metrics = {}
for modelName in model_names:
    model_metrics[modelName] = {}
    model_metrics[modelName][0] = 0.0
    model_metrics[modelName][1] = 0.0
    model_metrics[modelName][2] = 0.0
    model_metrics[modelName][3] = 0.0
    model_metrics[modelName][4] = 0.0

# calculate the metrics for all models
for rst in data_splits.keys():
    for model in model_names:
        metric = getMetrics(data_splits[rst][3], data_splits[rst][1], model_res[model][rst][0], model_res[model][rst][1])
        model_metrics[model][0] += metric[0]
        model_metrics[model][1] += metric[1]
        model_metrics[model][2] += metric[2]
        model_metrics[model][3] += metric[3]
        model_metrics[model][4] += metric[4]

# calculate the averages for each model
model_averages = {}
for model in model_names:
    model_averages[model] = {}
    model_averages[model][0] = model_metrics[model][0]/data_splits.keys().__len__()
    model_averages[model][1] = model_metrics[model][1]/data_splits.keys().__len__()
    model_averages[model][2] = model_metrics[model][2]/data_splits.keys().__len__()
    model_averages[model][3] = model_metrics[model][3]/data_splits.keys().__len__()
    model_averages[model][4] = model_metrics[model][4]/data_splits.keys().__len__()
print(model_averages)


c:\Users\maxep\OneDrive\Documents\Github Repo\SeniorProject\TheBioBros\.conda\Lib\site-packages\sklearn\utils\deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
c:\Users\maxep\OneDrive\Documents\Github Repo\SeniorProject\TheBioBros\.conda\Lib\site-packages\sklearn\utils\deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
c:\Users\maxep\OneDrive\Documents\Github Repo\SeniorProject\TheBioBros\.conda\Lib\site-packages\sklearn\utils\deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
c:\Users\maxep\OneDrive\Documents\Github Repo\SeniorProject\TheBioBros\.conda\Lib\site-packages\sklearn\utils\deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.wa

{'dt': {0: np.float64(0.9230769230769211), 1: 0.9000000000000008, 2: 0.7777777777777771, 3: 1.0, 4: 0.875}, 'xgb': {0: np.float64(1.0), 1: 0.9000000000000008, 2: 0.7777777777777771, 3: 1.0, 4: 0.875}, 'rf': {0: np.float64(0.9890109890109915), 1: 0.9000000000000008, 2: 0.7777777777777771, 3: 1.0, 4: 0.875}, 'gb': {0: np.float64(0.9230769230769211), 1: 0.9000000000000008, 2: 0.7777777777777771, 3: 1.0, 4: 0.875}, 'lr2': {0: np.float64(0.9890109890109915), 1: 0.9000000000000008, 2: 0.7777777777777771, 3: 1.0, 4: 0.875}, 'lr': {0: np.float64(0.9890109890109915), 1: 0.9000000000000008, 2: 0.7777777777777771, 3: 1.0, 4: 0.875}, 'knn': {0: np.float64(0.9175824175824185), 1: 0.9000000000000008, 2: 0.7777777777777771, 3: 1.0, 4: 0.875}, 'lsv': {0: np.float64(0.9780219780219755), 1: 0.9000000000000008, 2: 0.8571428571428572, 3: 0.8571428571428572, 4: 0.8571428571428572}, 'lgb': {0: np.float64(0.9890109890109915), 1: 0.9000000000000008, 2: 0.7777777777777771, 3: 1.0, 4: 0.875}, 'lre': {0: np.floa